# BE/Bi 103 Homework 7
###  Credits: 
A) John: mh_step() function

Jared: mh_sampler() function

B) John

In [1]:
import numpy as np
import pandas as pd
import numba
import scipy.stats as st

## Problem 7.1: Writing your own MCMC sampler (60 pts)

**a)** Write your own MCMC sampler that employs a Metropolis-Hastings algorithm to sample continuous parameters (i.e., it does not need to handle discrete parameters) that uses a Gaussian proposal distribution. Since you are sampling multiple parameters, your proposal distribution will be multi-dimensional. You can use a Gaussian proposal distribution with a diagonal covariance. In other words, you generate a proposal for each variable in the posterior independently.

You can organize your code how you like, but here is a suggestion.

* Write a function that takes (or rejects) a Metropolis-Hastings step. It should look something like the below (obviously where it does something instead of `pass`ing).

In [2]:
def mh_step(x, logpost, logpost_current, sigma, args=()):
    """
    Parameters
    ----------
    x : ndarray, shape (n_variables,)
        The present location of the walker in parameter space.
    logpost : function
        The function to compute the log posterior. It has call
        signature `logpost(x, *args)`.
    logpost_current : float
        The current value of the log posterior.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `logpost()` function.

    Returns
    -------
    output : ndarray, shape (n_variables,)
        The position of the walker after the Metropolis-Hastings
        step. If no step is taken, returns the inputted `x`.
    """
    # Turn sigma into a diagonal matrix so that it can be used as
    # the covariance matrix for numpy's multivariate guassian function
    cov = np.diag(sigma * sigma) # squared because variance is std. squared
    
    # Now I will choose the next step, x-prime
    xp = np.random.multivariate_normal(x, cov, 1)[0]
    
    # Compute the log-posterior at x-prime
    logpost_xp = logpost(xp, *args)
    
    # Calculate metropolis ratio
    r = np.exp(logpost_xp - logpost_current)
    
    if (r >= 1):
        return xp
    elif(np.random.rand() < r):
        return xp
    else:
        return x

* Write another function that calls that function over and over again to do the sampling. It should look something like the below. (Note that I am using `n_burn` as opposed to `n_warmup`, because you are just going to step as you normally would, and then "burn" the samples. This is in contrast to Stan, which tunes the stepping procedure duing the warm-up phase.)

In [3]:
def mh_sample(logpost, x0, sigma, args=(), n_burn=1000, n_steps=1000,
              variable_names=None):
    """
    Parameters
    ----------
    logpost : function
        The function to compute the log posterior. It has call
        signature `logpost(x, *args)`.
    x0 : ndarray, shape (n_variables,)
        The starting location of a walker in parameter space.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `logpost()` function.
    n_burn : int, default 1000
        Number of burn-in steps.
    n_steps : int, default 1000
        Number of steps to take after burn-in.
    variable_names : list, length n_variables
        List of names of variables. If None, then variable names
        are sequential integers.
    
    Returns
    -------
    output : DataFrame
        The first `n_variables` columns contain the samples.
        Additionally, column 'lnprob' has the log posterior value
        at each sample.
    """
    
    # Define variable names
    if variable_names == None:
        variable_names = list(range(len(x0)))
    
    # Define starting points
    x = x0
    logpost_current = logpost(x, *args)
        
    # Burn step    
    for c in range(n_burn):
        x = mh_step(x, logpost, logpost_current, sigma, args=args)
        logpost_current = logpost(x, *args)
    
    
    # Make arrays to take data
    samples = [0]*n_steps
    logpost_values = np.empty([len(range(n_steps))])
    
    # Do the sampling
    for k in range(n_steps):
        x = mh_step(x, logpost, logpost_current, sigma, args=args)
        logpost_current = logpost(x, *args)
        samples[k] = x
        logpost_values[k] = logpost_current
        
    
    # Convert arrays to dataframe    
    df =  pd.DataFrame(data=samples, columns = variable_names)
    df["lnprob"] = logpost_values
        
        
    return df

**b)** To test your code, we will get samples out of a known distribution. We will use a bivariate Gaussian with a mean of $\boldsymbol{\mu} = (10, 20)$ and covariance matrix of 

\begin{align}
\boldsymbol{\sigma} = \begin{pmatrix}
4 & -2 \\
-2 & 6
\end{pmatrix}
\end{align}

I have written the function to be unnormalized and JITted with [numba](https://numba.pydata.org) for optimal speed.

Do not be confused: In this test function we are sampling $\mathbf{x}$ out of $P(\mathbf{x}\mid \boldsymbol{\mu},\boldsymbol{\sigma})$. This is not sampling a posterior; it's just a test for your code. You will pass `log_test_distribution` as the `log_post` argument in the above functions.

In [4]:
mu = np.array([10.0, 20])
cov = np.array([[4, -2],[-2, 6]])
inv_cov = np.linalg.inv(cov)

@numba.jit(nopython=True)
def log_test_distribution(x, mu, inv_cov):
    """
    Unnormalized log posterior of a multivariate Gaussian.
    """
    return -np.dot((x-mu), np.dot(inv_cov, (x-mu))) / 2

If you compute the means and covariance (using `np.cov()`) of your samples, you should come close to the inputed means and covariance. You might also want to plot your samples using `bebi103.viz.corner()` to make sure everything makes sense.

<br />

You may want to add in some logic to your Metropolis-Hastings sampler to enable *tuning*. This is the process of automatically adjusting the $\sigma$ in the proposal distribution such that the acceptance rate is desirable. The target acceptance rate is about 0.4. The developers of [PyMC3](https://github.com/pymc-devs/pymc3) use the scheme below, which is reasonable.

|Acceptance rate|Standard deviation adaptation|
|:---:|:-------------------:|
| < 0.001        |$\times$ 0.1|
|< 0.05         |$\times$ 0.5|
|< 0.2          |$\times$ 0.9|
|> 0.5          |$\times$ 1.1|
|> 0.75         |$\times$ 2|
|> 0.95         |$\times$ 10|

Be sure to test your code to demonstrate that it works.

In [5]:
# Initial guess for MCMC
x_0 = np.array([1,1])
# Standard Deviations for proposal dist.
s = np.array([1,1])
df_mcmc_test = mh_sample(log_test_distribution, 
                         x_0, 
                         s, 
                         args = (mu, inv_cov), 
                         variable_names = ["var_1", "var_2"],
                         n_steps = 4000)
df_mcmc_test.head()

,var_1,var_2,lnprob
0,9.736044,19.771434,-0.021708
1,10.144924,20.282549,-0.015229
2,9.962231,20.007777,-0.000191
3,9.759013,19.582742,-0.036177
4,9.682013,20.237301,-0.013253


It looks like the sampling worked! Let's see how the parameters compare to the real distribution. 

In [6]:
print("MCMC mean of var_1: %.4f    Actual mean of var1: 10.0" % 
      np.mean(df_mcmc_test["var_1"].values))
print("MCMC mean of var_2: %.4f    Actual mean of var2: 20.0" % 
      np.mean(df_mcmc_test["var_2"].values))
covariance = np.cov(df_mcmc_test["var_1"].values, df_mcmc_test["var_2"].values)
print("")
print(
"MCMC covariance matrix: %.4f %.4f \n                       %.4f %.4f" 
    %(covariance[0][0],
      covariance[0][1],
      covariance[1][0],
      covariance[1][1]))
print("")
print(
"Actual covariance matrix: %.4f %.4f \n                         %.4f %.4f"
    %(cov[0][0],
      cov[0][1],
      cov[1][0],
      cov[1][1]))

MCMC mean of var_1: 9.9744    Actual mean of var1: 10.0
MCMC mean of var_2: 20.0402    Actual mean of var2: 20.0

MCMC covariance matrix: 4.3028 -2.2645 
                       -2.2645 6.4868

Actual covariance matrix: 4.0000 -2.0000 
                         -2.0000 6.0000


These numbers are pretty variable when there is a limited number of samples. Perhaps this can be improved via acceptance rate optimization. 

We are interested in calculating the acceptance rate of moves in our MCMC sampler. I will write a function for this. 

In [7]:
def calc_acceptance_rate(output_df):
    """Determines the acceptance rate of an MCMC sampler within the passed
    dataframe containing MCMC samples. """
    num_rejections = 0
    last_row = np.zeros(len(output_df.loc[0].values))
    for row in output_df.iterrows():
        row = row[1].values
        if (row == last_row).all():
            num_rejections += 1
        last_row = row
    return (len(output_df.index) - num_rejections) / len(output_df.index)

I will do a quick check to make sure that this function is sensible. 

In [8]:
calc_acceptance_rate(df_mcmc_test)

0.768

It looks like we are being slightly too forgiving in the number of steps we are accepting! I will now write a small helper function that will determine the standard deviation correction, essentially from a table. 

In [9]:
def find_std_dev_correction(acceptance_rate):
    """Returns the necessary multiplicative correction to the standard deviation
    of the proposal distribution given an acceptance rate for the metropolis-
    hastings MCMC sampler. """
    if acceptance_rate < .001:
        return 0.1
    elif acceptance_rate < 0.05:
        return 0.5
    elif acceptance_rate < 0.2:
        return 0.9
    elif acceptance_rate < 0.5:
        return 1
    elif acceptance_rate < 0.75:
        return 1.1
    elif acceptance_rate < 0.95:
        return 2
    else:
        return 10

Now I want to re-write my sampling function to ensure that the acceptance rate is valid. I will do a trial of sampling after the burn step to ensure there is a good acceptance rate, and repeat trials until the acceptance rate is acceptable. 

In [10]:
def mh_sample_optimize_acceptance(logpost, 
                                  x0, 
                                  sigma, 
                                  args=(), 
                                  n_burn=1000, 
                                  n_steps=1000, 
                                  n_opt_acc = 1000,
                                  variable_names=None):
    """New sampling function that includes optimization of sigma
    with respect to the acceptance rate. n_opt_acc indicates the number
    of steps that should be used to optimize, but all other parameters
    are identical to those used in mh_sample()"""
    
    # Define variable names
    if variable_names == None:
        variable_names = list(range(len(x0)))
    
    # Define starting points
    x = x0
    logpost_current = logpost(x, *args)
        
    # Burn step    
    for c in range(n_burn):
        x = mh_step(x, logpost, logpost_current, sigma, args=args)
        logpost_current = logpost(x, *args)
    
    # Starting optimization!!
    iterations = 0
    while(True):
        # Make arrays to take optimization data
        samples_opt = [0]*n_opt_acc
        logpost_values_opt = np.empty([len(range(n_opt_acc))])

        # Do the sampling
        for k in range(n_opt_acc):
            x = mh_step(x, logpost, logpost_current, sigma, args=args)
            logpost_current = logpost(x, *args)
            samples_opt[k] = x
            logpost_values_opt[k] = logpost_current

        # Convert arrays to dataframe    
        df_opt =  pd.DataFrame(data=samples_opt, columns = variable_names)
        df_opt["lnprob"] = logpost_values_opt
        
        # Calculate acceptance rate and optimize sigma
        rate = calc_acceptance_rate(df_opt)
        opt_factor = find_std_dev_correction(rate)
        sigma *= opt_factor
        
        # If the acceptance rate was way off, repeat this process again. 
        # Otherwise, we can exit the loop
        if (opt_factor == 1):
            break
            
        # This program has the potential for an infinite loop. I don't want
        # this to happen, so if this iterates 10 times I'll print a warning
        # statement and stop optimizing. 
        iterations += 1
        if iterations >= 10:
            print("WARNING: Acceptance rate did not converge!")
            break
    
    # Make arrays to take data
    samples = [0]*n_steps
    logpost_values = np.empty([len(range(n_steps))])
    
    # Do the sampling
    for k in range(n_steps):
        x = mh_step(x, logpost, logpost_current, sigma, args=args)
        logpost_current = logpost(x, *args)
        samples[k] = x
        logpost_values[k] = logpost_current
        
    
    # Convert arrays to dataframe    
    df =  pd.DataFrame(data=samples, columns = variable_names)
    df["lnprob"] = logpost_values

    return df

Let's re-run our previous check and see if the results are improved. 

In [11]:
# Initial guess for MCMC
x_0 = np.array([1,1])
# Standard Deviations for proposal dist.
s = np.array([1.,1.])
df_mcmc_opt  = mh_sample_optimize_acceptance(log_test_distribution, 
                                             x_0, 
                                             s, 
                                             args = (mu, inv_cov), 
                                             variable_names = ["var_1", "var_2"],
                                             n_steps = 4000)

print("MCMC mean of var_1: %.4f    Actual mean of var1: 10.0" % 
      np.mean(df_mcmc_opt["var_1"].values))
print("MCMC mean of var_2: %.4f    Actual mean of var2: 20.0" % 
      np.mean(df_mcmc_opt["var_2"].values))
covariance = np.cov(df_mcmc_opt["var_1"].values, df_mcmc_opt["var_2"].values)
print("")
print(
"MCMC covariance matrix: %.4f %.4f \n                       %.4f %.4f" 
    %(covariance[0][0],
      covariance[0][1],
      covariance[1][0],
      covariance[1][1]))
print("")
print(
"Actual covariance matrix: %.4f %.4f \n                         %.4f %.4f"
    %(cov[0][0],
      cov[0][1],
      cov[1][0],
      cov[1][1]))
print("Acceptance Rate: %.3f" % calc_acceptance_rate(df_mcmc_opt))

MCMC mean of var_1: 10.1283    Actual mean of var1: 10.0
MCMC mean of var_2: 19.9409    Actual mean of var2: 20.0

MCMC covariance matrix: 4.0311 -2.0677 
                       -2.0677 5.8159

Actual covariance matrix: 4.0000 -2.0000 
                         -2.0000 6.0000
Acceptance Rate: 0.539


It's a little hard to see whether the results have actually improved, but the acceptance rate has certainly been optimized. Our MCMC sampler works as predicted. 